In [1]:
!nvidia-smi

Sun Mar 23 15:44:25 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [42]:
!apt update
!apt install libcudnn9-cuda-12
!pip install nvidia-cublas-cu12 nvidia-cudnn-cu12==9.*

!export LD_LIBRARY_PATH=`python3 -c 'import os; import nvidia.cublas.lib; import nvidia.cudnn.lib; print(os.path.dirname(nvidia.cublas.lib.__file__) + ":" + os.path.dirname(nvidia.cudnn.lib.__file__))'`


Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,769 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [3,962 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 P

In [2]:
!pip3 install faster-whisper ctranslate2 datasets sacrebleu -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.6/38.6 MB 20.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 11.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 82.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 12.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8

In [24]:
from datasets import load_dataset, Dataset, Audio
import numpy as np

In [ ]:
import os
cache_dir = "/content/huggingface_cache"
os.makedirs(cache_dir, exist_ok=True)

# Set ALL Hugging Face related cache directories
os.environ["TRANSFORMERS_CACHE"] = os.path.join(cache_dir, "transformers")
os.environ["HF_DATASETS_CACHE"] = os.path.join(cache_dir, "datasets")
os.environ["HF_HOME"] = os.path.join(cache_dir, "hf_home")
os.environ["HF_ASSETS_CACHE"] = os.path.join(cache_dir, "assets")
os.environ["HUGGINGFACE_HUB_CACHE"] = os.path.join(cache_dir, "hub")
os.environ["HF_MODULES_CACHE"] = os.path.join(cache_dir, "modules")

# Create all directories
for dir_path in [os.environ["TRANSFORMERS_CACHE"],
                os.environ["HF_DATASETS_CACHE"],
                os.environ["HF_HOME"],
                os.environ["HF_ASSETS_CACHE"],
                os.environ["HUGGINGFACE_HUB_CACHE"],
                os.environ["HF_MODULES_CACHE"]]:
    os.makedirs(dir_path, exist_ok=True)

# Force datasets to use the new cache
from datasets import config
config.HF_DATASETS_CACHE = os.environ["HF_DATASETS_CACHE"]

# Load the Dataset

In [30]:
dataset_name = "kreasof-ai/be-en-IWSLT2025"
test_dataset = load_dataset(dataset_name, trust_remote_code=True, split="test")

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

IterableDatasetDict({
    train: IterableDataset({
        features: ['audio', 'sentence', 'translation', 'speaker_id'],
        num_shards: 41
    })
    val: IterableDataset({
        features: ['audio', 'sentence', 'translation', 'speaker_id'],
        num_shards: 2
    })
    test: IterableDataset({
        features: ['audio', 'sentence', 'translation', 'speaker_id'],
        num_shards: 2
    })
})

In [31]:
test_dataset

IterableDataset({
    features: ['audio', 'sentence', 'translation', 'speaker_id'],
    num_shards: 2
})

In [32]:
for sample in test_dataset:
    print(sample["audio"])  # Access the audio column
    break  # Just to check the first example

{'path': '15_7474_3_152_02_211030-180248_bem_8cc_elicit_1.wav', 'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
       -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]), 'sampling_rate': 16000}


# Testing with first 10 data

In [60]:
# Get the first 10 examples
test_dataset = list(dataset["test"].take(10))

In [61]:
test_dataset = Dataset.from_list(test_dataset)

In [62]:
test_dataset = test_dataset.cast_column("audio", Audio())

In [63]:
test_dataset["audio"]

[{'path': None,
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([ 0.        ,  0.        ,  0.        , ..., -0.01599121,
         -0.01629639, -0.01654053]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([ 0.00000000e+00, -3.05175781e-05, -3.05175781e-05, ...,
         -1.12915039e-03, -8.23974609e-04, -4.88281250e-04]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([0.        , 0.        , 0.        , ..., 0.00061035, 0.00204468,
         0.00100708]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([ 0.        ,  0.        ,  0.        , ...,  0.00094604,
          0.00024414, -0.00039673]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([0.        , 0.        , 0.        , ..., 0.00021362, 0.00021362,
         0.        ]),
  'sampling_rate': 16000},
 {'path': None,
  'array': array([

# Create Arrays for audio

In [67]:
references = test_dataset["translation"]
print(references) # Check if the references is list

['he is wearing glasses as well', 'A dog with a wide nose is running on the lawns of the football ground', 'She has her leg in the air attempting to break a board.', 'There are also small vehicles that they use in cold places behind them.', 'Of the men on the roof of the house, ine is wearing a long sleeved grey shirt and a blue trousers.', 'These on their shoes are for sliding when the to play on the ice.', 'A woman is walking in the road with a child on her showders.', "I have seen it seems there are different games being played that's why others are looking the other way while others are watching volleyball.", 'A woman is holding food on a plate inside a shop.', 'Or maybe someone has made him upset.']


In [69]:
audio_arrays = [a["array"].astype(np.float32) for a in test_dataset["audio"]]

In [70]:
audio_arrays

[array([ 0.0000000e+00,  0.0000000e+00,  0.0000000e+00, ...,
        -9.1552734e-05, -6.1035156e-05, -6.1035156e-05], dtype=float32),
 array([ 0.        ,  0.        ,  0.        , ..., -0.01599121,
        -0.01629639, -0.01654053], dtype=float32),
 array([ 0.0000000e+00, -3.0517578e-05, -3.0517578e-05, ...,
        -1.1291504e-03, -8.2397461e-04, -4.8828125e-04], dtype=float32),
 array([0.        , 0.        , 0.        , ..., 0.00061035, 0.00204468,
        0.00100708], dtype=float32),
 array([ 0.        ,  0.        ,  0.        , ...,  0.00094604,
         0.00024414, -0.00039673], dtype=float32),
 array([0.        , 0.        , 0.        , ..., 0.00021362, 0.00021362,
        0.        ], dtype=float32),
 array([ 0.0000000e+00, -3.0517578e-05, -3.0517578e-05, ...,
        -1.5258789e-04, -3.3569336e-04, -6.1035156e-04], dtype=float32),
 array([0.        , 0.        , 0.        , ..., 0.00100708, 0.00106812,
        0.00119019], dtype=float32),
 array([ 0.        ,  0.        ,  0

In [ ]:
assert len(audio_arrays) == len(references)

# Preprocess Dataset

# Cascaded

## transcribe

In [40]:
model_id = "kreasof-ai/whisper-small-be2en"
output_dir="ct2-whisper-small-transcription"

!ct2-transformers-converter \
--model {model_id} \
--output_dir {output_dir} \
--quantization float16 \
--copy_files tokenizer_config.json\
--force

config.json: 100% 1.38k/1.38k [00:00<00:00, 8.32MB/s]
2025-03-23 15:57:43.843091: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742745463.863009    6407 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742745463.869007    6407 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 15:57:43.889403: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
model.safetensors: 100% 967M/967M [00:39<00:00, 24.6MB/s]
gener

In [41]:
from faster_whisper import WhisperModel

model_name = "ct2-whisper-small-transcription" # ct2-whisper-xxx

model = WhisperModel(model_name, device="cuda", compute_type="float16")

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

In [71]:
test_dataset[0]

{'audio': {'path': None,
  'array': array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
         -9.15527344e-05, -6.10351562e-05, -6.10351562e-05]),
  'sampling_rate': 16000},
 'sentence': 'nafwala na amakalashi ku menso',
 'translation': 'he is wearing glasses as well',
 'speaker_id': 152}

In [72]:
tgt_lang = "be"

transcriptions = []
segments, info = model.transcribe(audio_arrays[0],
                                beam_size=5,
                                language=tgt_lang,
                                vad_filter=True
                                )
transcription = " ".join([segment.text.strip() for segment in segments])
transcriptions.append(transcription)

In [73]:
print(transcriptions)

['nafwala na amakalashi ku menso']


In [74]:
from tqdm.auto import tqdm

tgt_lang = "be"

transcriptions = []

for audio_array in tqdm(audio_arrays, total=len(audio_arrays)):
    segments, info = model.transcribe(audio_array,
                                        beam_size=5,
                                        language=tgt_lang,
                                        vad_filter=True)
    transcription = " ".join([segment.text.strip() for segment in segments])
    transcriptions.append(transcription)

  0%|          | 0/10 [00:00<?, ?it/s]

In [75]:
print(*transcriptions[:20], sep="\n")

nafwala na amakalashi ku menso
Imbwa imo iya fonka pamona ilebutuka pa lunkoto lwa mucibansa
Akamwana nakemya ukulu mumuulu ukulwisha ukutoba akapulana.
Kunuma yabo kuli notuma motoka tulya babonfya mu ncende iyatalala nge iyi baliko.
abaume Bali pamutenge ya ng'anda umo afwele ishati lya mitomito ilyamaboko ayatali elyo netoloshi lya makumbimakumbi
Ifi bafwele kunsapato fyakutelela nga ba mukwangala umu mwine muli ice.
Namayo alenda mumusebo nabika nomwana pamabeya.
Nimbona, iyi incende balateyelako ifyangalo ifyapusana pusana emulandu wine kuli bambi abaletamba umupila wakuminwe elyo bambi baposele amano mukutambeba ifyangalo fimbi fi.
Namayo naikata ifyakulya pambale mukati ke tuka.
Nangu limbi kuli bamo abamufulwishe.


## MT (NLLB)

In [51]:
mt_model_name = "kreasof-ai/nllb-IWSLT2025-be-en"
output_mt = "ct2-nllb-be-en"
!ct2-transformers-converter --model {mt_model_name} --quantization float16 --output_dir {output_mt}

config.json: 100% 897/897 [00:00<00:00, 4.02MB/s]
2025-03-23 16:02:43.510406: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1742745763.533550    8434 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1742745763.540090    8434 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-03-23 16:02:43.562376: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
model.safetensors: 100% 2.46G/2.46G [01:47<00:00, 22.8MB/s]
generat

In [52]:
!wget https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model

--2025-03-23 16:05:19--  https://s3.amazonaws.com/opennmt-models/nllb-200/flores200_sacrebleu_tokenizer_spm.model
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.204.144, 52.216.153.62, 16.15.193.31, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.204.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4852054 (4.6M) [binary/octet-stream]
Saving to: ‘flores200_sacrebleu_tokenizer_spm.model’

flores200_sacrebleu 100%[===================>]   4.63M   944KB/s    in 9.1s    

2025-03-23 16:05:29 (520 KB/s) - ‘flores200_sacrebleu_tokenizer_spm.model’ saved [4852054/4852054]



In [56]:
import os

directory = "/content/"

ct2_model_name = output_mt

ct_model_path = os.path.join(directory, ct2_model_name)
sp_model_path = os.path.join(directory, "flores200_sacrebleu_tokenizer_spm.model")

In [57]:
import ctranslate2
import sentencepiece as spm
import torch

device = "cuda" if torch.cuda.is_available() else "cpu"

sp = spm.SentencePieceProcessor()
sp.load(sp_model_path)

translator = ctranslate2.Translator(ct_model_path, device=device)

In [76]:
# Translate

# src_lang = "ben_Beng"
# tgt_lang = "eng_Latn"

src_lang = "bem_Latn"
tgt_lang = "eng_Latn"

beam_size = 5

source_sentences = transcriptions

source_sents = [sent.strip() for sent in source_sentences]
target_prefix = [[tgt_lang]] * len(source_sents)

# Subword the source sentences
source_sents_subworded = sp.encode_as_pieces(source_sents)
source_sents_subworded = [[src_lang] + sent + ["</s>"] for sent in source_sents_subworded]

# Translate the source sentences
translations = translator.translate_batch(source_sents_subworded,
                                          batch_type="tokens",
                                          max_batch_size=2024,
                                          beam_size=beam_size,
                                          target_prefix=target_prefix)
translations = [translation.hypotheses[0] for translation in translations]

# Desubword the target sentences
translations_desubword = sp.decode(translations)
translations_desubword = [sent[len(tgt_lang):].strip() for sent in translations_desubword]

print(*translations_desubword[:10], sep="\n")

he is wearing glasses on his face
A dog with a wrinkled nose is running on the grass in the ground.
A child has lifted up his leg in an attempt to break the knot.
Behind them are vehicles that they use in cold places like this one they are in.
A man is on the roof of a house wearing a grey long sleeved shirt and a blue trousers.
These shoes they are wearing are for sliding on ice.
A woman is walking on the road with a child on her shoulders.
I have seen. This is a place where different games are played and that is why there are people watching basketball and others are paying attention to watching other games.
A woman is holding food on a plate inside a shop.
Or maybe someone has upset him.


In [77]:
import sacrebleu

translations = translations_desubword

# Calculate BLEU
bleu = sacrebleu.corpus_bleu(translations, [references])
bleu = round(bleu.score, 2)

# Calculate CHRF
chrf = sacrebleu.corpus_chrf(translations, [references], word_order=2)  # for chrF++ word_order=2
chrf = round(chrf.score, 2)

print(model_name)
print("BLEU\tChrF++")
print(f"{bleu}\t{chrf}")

ct2-whisper-small-transcription
BLEU	ChrF++
37.38	59.34
